In [1]:
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Load data
data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Prepare data
texts = []
labels_raw = []
for i in data:
    texts.append(i["text"])
    labels_raw.append(i["label_int"])

labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

# Create datasets
train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})
val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

400


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 80
    })
})


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EvalPrediction
import evaluate
import numpy as np
from tqdm.auto import tqdm

# Set seed for reproducibility
torch.manual_seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer and model
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming raw_datasets is a DatasetDict with "train" and "validation" splits
# and each example has "text" and "label" fields

# Function to tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization to the datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Load the model
num_labels = len(set(raw_datasets["text"]["label"]))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model = model.to(device)  # Move model to GPU

# Define metrics for evaluation
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Add CUDA specific arguments
    fp16=True,  # Use mixed precision training
    no_cuda=False,  # Use CUDA
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Custom training loop to report training loss, evaluation loss, and accuracy per epoch
for epoch in range(10):
    print(f"\nEpoch {epoch+1}/10")
    
    # Training
    train_loss = 0
    model.train()
    train_dataloader = trainer.get_train_dataloader()
    
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to GPU
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        trainer.optimizer.step()
        trainer.lr_scheduler.step()
        trainer.optimizer.zero_grad()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss:.4f}")
    
    # Evaluation
    model.eval()
    eval_loss = 0
    eval_preds = []
    eval_labels = []
    eval_dataloader = trainer.get_eval_dataloader()
    
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to GPU
            outputs = model(**batch)
            loss = outputs.loss
            eval_loss += loss.item()
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            eval_preds.extend(predictions.cpu().numpy())
            eval_labels.extend(batch["labels"].cpu().numpy())
    
    avg_eval_loss = eval_loss / len(eval_dataloader)
    accuracy = accuracy_metric.compute(predictions=eval_preds, references=eval_labels)["accuracy"]
    
    print(f"Evaluation Loss: {avg_eval_loss:.4f}")
    print(f"Evaluation Accuracy: {accuracy:.4f}")

Using device: cuda


Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

KeyError: "Column label not in the dataset. Current columns in the dataset: ['text', 'labels']"

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./batterybert-cased-abstract_finetuned_400_2")
tokenizer.save_pretrained("./batterybert-cased-abstract_finetuned_400_2")

In [ ]:
# For inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to GPU
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.cpu().numpy()  # Move predictions back to CPU for numpy conversion

In [ ]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = predict(val_data["text"][i])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_fp = 0
openai_fn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1

print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
7	0	1	0	0.9998679161071777
14	0	0	1	0.9926759600639343
21	0	1	0	0.9998675584793091
26	1	1	0	0.9979070425033569
27	1	1	0	0.9998754262924194
34	0	1	0	0.9998750686645508
36	0	1	0	0.9998751878738403
39	1	1	0	0.9998252987861633
42	1	1	0	0.960843563079834
45	1	1	0	0.9922792911529541
47	1	1	0	0.999674916267395
53	0	1	0	0.9998691082000732
75	0	0	1	0.993808925151825
79	0	1	0	0.9998757839202881
80	0	1	0	0.9997312426567078
83	0	0	1	0.9937708973884583
86	1	1	0	0.9607234597206116
87	1	1	0	0.9994949102401733
89	1	1	0	0.9998729228973389
99	0	1	0	0.9998770952224731
OpenAI False Positive: 8
OpenAI False Negative: 0
Bert False Positive: 3
Bert False Negative: 9
